In [1]:
%%javascript
// Шоткат для перемещения к исполняемой
Jupyter.keyboard_manager.command_shortcuts.add_shortcut('Alt-I', {
    help : 'Go to Running cell',
    help_index : 'zz',
    handler : function (event) {
        setTimeout(function() {
            // Find running cell and click the first one
            if ($('.running').length > 0) {
                //alert("found running cell");
                $('.running')[0].scrollIntoView();
            }}, 250);
        return false;
    }
});

<IPython.core.display.Javascript object>

In [2]:
import sys
import os
import warnings
os.environ['OPENBLAS_NUM_THREADS'] = '1'
warnings.filterwarnings('ignore')

In [3]:
import tqdm
import pickle 
import random
import seaborn as sns
import matplotlib.pyplot as plt
from fuzzywuzzy import fuzz

In [4]:
import pandas as pd
import numpy as np
import time
import pyarrow.parquet as pq
import pyarrow as pa
import scipy
import implicit
import bisect
import sklearn.metrics as m
from catboost import CatBoostClassifier, CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split
from sklearn.calibration import calibration_curve, CalibratedClassifierCV
from sklearn.preprocessing import LabelEncoder

In [5]:
SPLIT_SEED = 666
LOCAL_DATA_PATH = 'E:/mts'
DATA_FILE = 'competition_data_final_pqt'
TARGET_FILE = 'public_train.pqt'
SUBMISSION_FILE = 'submit_2.pqt'

In [6]:
id_to_submit = pq.read_table(f'{LOCAL_DATA_PATH}/{SUBMISSION_FILE}').to_pandas()
id_to_submit

,user_id
221301,221301
31271,31271
211594,211594
253119,253119
192578,192578
...,...
145307,415284
145308,415285
145309,415286
145312,415306


In [7]:
data = pq.read_table(f'{LOCAL_DATA_PATH}/{DATA_FILE}')
pd.DataFrame([(z.name, z.type) for z in data.schema], columns = [['field', 'type']])

,field,type
0,region_name,string
1,city_name,string
2,cpe_manufacturer_name,string
3,cpe_model_name,string
4,url_host,string
5,cpe_type_cd,string
6,cpe_model_os_type,string
7,price,double
8,date,date32[day]
9,part_of_day,string


In [8]:
targets = pq.read_table(f'{LOCAL_DATA_PATH}/{TARGET_FILE}')
pd.DataFrame([(z.name, z.type) for z in targets.schema], columns = [['field', 'type']])

,field,type
0,age,double
1,is_male,string
2,user_id,int64
3,__index_level_0__,int64


In [9]:
OBJ_COLS = [
            'region_name', 
            'city_name', 
            'cpe_manufacturer_name',
            'cpe_model_name',
            'url_host',
            'cpe_type_cd',
            'cpe_model_os_type', 
#             'price', 
#             'date',
            'part_of_day', 
#             'request_cnt',
#             'user_id'
           ]

DIGIT_COLS = [
            'price', 
#             'date',
            'request_cnt',
#             'user_id'
            ]

In [10]:
url_set = set(data.select(['url_host']).to_pandas()['url_host'])
print(f'{len(url_set)} urls')
url_dict = {url: idurl for url, idurl in zip(url_set, range(len(url_set)))}

199683 urls


In [11]:
usr_set = set(data.select(['user_id']).to_pandas()['user_id'])
print(f'{len(usr_set)} users')
usr_dict = {usr: user_id for usr, user_id in zip(usr_set, range(len(usr_set)))}

415317 users


In [17]:
values = np.array(data.select(['request_cnt_sum']).to_pandas()['request_cnt_sum'])

KeyError: 'Field "request_cnt_sum" does not exist in table schema'

In [ ]:
data[:4].to_pandas()

In [ ]:
unique_obj_dict = {}
for col in OBJ_COLS:
    unique_obj_dict[col] = dict(zip(set(data.to_pandas()[col]), range(len(set(data.to_pandas()[col])))))

In [ ]:
unique_obj_dict = {}
for col in OBJ_COLS:
    unique_obj_dict[col] = set(data)
    if col not in unique_obj_dict:
        unique_obj_dict[col] = set(chunk[col])
    else:
        unique_obj_dict[col] = unique_obj_dict[col].union(set(chunk[col]))

In [ ]:
# %%script false
### Сбор уникальных значений столбцов в словарь для преобразования датасета ###

# Wall time: 8min 34s

# %%time
unique_obj_dict = {}
i = 0
for chunk in pd.read_parquet(f'{LOCAL_DATA_PATH}/{DATA_FILE}'):
        i += 1
        print(f'chunk_{i}')
        for col in OBJ_COLS:
            print(f'col_{col}')
            if col not in unique_obj_dict:
                unique_obj_dict[col] = set(chunk[col])
            else:
                unique_obj_dict[col] = unique_obj_dict[col].union(set(chunk[col]))

In [ ]:
data[:4].to_pandas()